# Benchmark Cognitive Services Azure (Text Analytics)



In [24]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import pandas as pd
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
key = os.environ["COGNITIVE_AZURE_KEY"]
endpoint = os.environ["COGNITIVE_AZURE_ENDPOINT"]

In [3]:
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [5]:
def sentiment_analysis_example(client):

    documents = ["I had the best day of my life. I wish you were there with me."]
    response = client.analyze_sentiment(documents=documents)[0]
    print("Document Sentiment: {}".format(response.sentiment))
    print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
        response.confidence_scores.positive,
        response.confidence_scores.neutral,
        response.confidence_scores.negative,
    ))
    for idx, sentence in enumerate(response.sentences):
        print("Sentence: {}".format(sentence.text))
        print("Sentence {} sentiment: {}".format(idx+1, sentence.sentiment))
        print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
            sentence.confidence_scores.positive,
            sentence.confidence_scores.neutral,
            sentence.confidence_scores.negative,
        ))
          
sentiment_analysis_example(client)

Document Sentiment: positive
Overall scores: positive=1.00; neutral=0.00; negative=0.00 

Sentence: I had the best day of my life.
Sentence 1 sentiment: positive
Sentence score:
Positive=1.00
Neutral=0.00
Negative=0.00

Sentence: I wish you were there with me.
Sentence 2 sentiment: neutral
Sentence score:
Positive=0.21
Neutral=0.77
Negative=0.02



In [6]:
def send_request(document):
    if len(document) > 10:
        all_doc_send=[]
        response=[]
        range_10 = [i for i in range(0,len(document)+1,10)]
        for i in range(1,len(range_10)):
            all_doc_send.append(document[range_10[i - 1 ]:range_10[i]])
        for doc in all_doc_send:
            response.extend(client.analyze_sentiment(documents=doc))
    else:
        response = client.analyze_sentiment(documents=document)
    return response

def format_response(response):
    for resp in response:
        print("Document Sentiment: {}".format(resp.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            resp.confidence_scores.positive,
            resp.confidence_scores.neutral,
            resp.confidence_scores.negative,
        ))
        
def get_pred(resp,get_neutral=False):
    positive = resp.confidence_scores.positive
    neutral = resp.confidence_scores.neutral
    negative = resp.confidence_scores.negative
    if get_neutral :
        if (positive > neutral) & (positive > negative):
            return "good"
        elif (negative > neutral) & (negative > positive):
            return "bad"
        else:
            return "neutral"
    else:
        if positive > negative:
            return "good"
        else:
            return "bad"
    
def get_response_prediction(response,get_neutral=False):
    dic={
        "good_score":[],
        "neutral_score":[],
        "bad_score":[],
        "prediction":[],
    }
    for resp in response:
        dic["good_score"].append(resp.confidence_scores.positive)
        dic["neutral_score"].append(resp.confidence_scores.neutral)
        dic["bad_score"].append(resp.confidence_scores.negative)
        dic["prediction"].append(get_pred(resp,get_neutral=get_neutral))
    return dic

# Benchmark azure cognitive

In [51]:
train_sample = pd.read_csv("./data/sample_final_dataset.csv",index_col=0)
print("Dimensionnalité :",train_sample.shape)
train_sample.head()

Dimensionnalité : (10000, 2)


,text,sentiment
0,I LOVE @Health4UandPets u guys r the best!!,good
1,im meeting up with one of my besties tonight! ...,good
2,"@DaRealSunisaKim Thanks for the Twitter add, S...",good
3,Being sick can be really cheap when it hurts t...,good
4,@LovesBrooklyn2 he has that effect on everyone,good


In [52]:
# on stratify pour ne prendre que 4000 tweets
_,train_sample_ = train_test_split(train_sample,test_size=0.4,stratify=train_sample["sentiment"],random_state=42) 

In [53]:
train_sample_.shape

(4000, 2)

In [54]:
train_sample_["sentiment"].value_counts()

bad     2000
good    2000
Name: sentiment, dtype: int64

In [55]:
document_send = train_sample_["text"].tolist()
y_true = train_sample_["sentiment"].tolist()

In [11]:
# %%time
# dic_response = get_response_prediction(send_request(document_send))
# joblib.dump(dic_response,"./data/cognitive_service_response_train.pkl")

CPU times: user 2.89 s, sys: 131 ms, total: 3.03 s
Wall time: 2min 2s


In [56]:
dic_response = joblib.load("./data/cognitive_service_response_train.pkl")

In [57]:
good_score = dic_response["good_score"]
neutral_score = dic_response["neutral_score"]
bad_score = dic_response["bad_score"]

In [58]:
lr = LogisticRegression()
feature_cogn = np.array([good_score,neutral_score,bad_score]).T
lr.fit(feature_cogn,y_true)

LogisticRegression()

In [59]:
lr.coef_

array([[ 1.27577746,  0.3113381 , -1.5870753 ]])

### Test sample

In [60]:
test_sample = pd.read_csv("./data/sample_test_dataset.csv",index_col=0)
print("Dimensionnalité :",test_sample.shape)
test_sample.head()

Dimensionnalité : (4000, 2)


,text,sentiment
0,Chillin @ the beach with my girl brit-brit wat...,good
1,@indraherlambang really wanna say that ure lik...,good
2,"@ambermac Gotta love Brad Pitt's performance, ...",good
3,Just realized that Matthew Sweet &amp; Susanna...,good
4,"@marksphone sorry, meant that to come from my ...",good


In [61]:
test_sample["sentiment"].value_counts()

bad     2000
good    2000
Name: sentiment, dtype: int64

## tous les tweets

Le coût de ce service est de 0.87€ pour 1000 requêtes, avec ce test nous somme à 4000 requêtes sois un total de 3.48€

In [62]:
document_send = test_sample["text"].tolist()
y_true = test_sample["sentiment"].tolist()

In [9]:
# dic_response = get_response_prediction(send_request(document_send))
# joblib.dump(dic_response,"./data/cognitive_service_response.pkl")

In [63]:
dic_response = joblib.load("./data/cognitive_service_response.pkl")

In [64]:
good_score = dic_response["good_score"]
neutral_score = dic_response["neutral_score"]
bad_score = dic_response["bad_score"]
feature_cogn = np.array([good_score,neutral_score,bad_score]).T

### Régression logistique pour obtenir les score bad & good

In [65]:
y_pred = lr.predict(feature_cogn)
y_pred_proba = lr.predict_proba(feature_cogn)

In [46]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

def show_result_binary(y_pred, y_pred_prob, y_true):
    print("accuracy score :",accuracy_score(y_true,y_pred))
    print("AUC score :",roc_auc_score(y_true,y_pred_prob[:,1]))
    cm = confusion_matrix(y_true,y_pred)
    print(classification_report(y_true,y_pred))
    return pd.DataFrame(cm,columns=["good","bad"],index=["good","bad"])

In [47]:
show_result_binary(y_pred,y_pred_proba,y_true)

accuracy score : 0.71225
AUC score : 0.778609125
              precision    recall  f1-score   support

         bad       0.78      0.59      0.67      2000
        good       0.67      0.83      0.74      2000

    accuracy                           0.71      4000
   macro avg       0.73      0.71      0.71      4000
weighted avg       0.73      0.71      0.71      4000



,good,bad
good,1183,817
bad,334,1666


Voici les résultats du service cognitif d'azure, on vois ici que les résultats sont globalement assez bon notament pour la détection des bad tweet avec une précision de 0.77 des bad tweet le score AUC est ici de 0.7786 qui est un score correct mais pas très performant.
Pour résumer le service cognitif d'azure est un outil très généraliste et permet d'avoir un service de détéction des sentiment rapidement et globalement assez efficace, cependant, les résultats sont loin d'être optimaux et pour des frais d'utilisation assez consequent pour une utilisation soutenue du service.